In [1]:
import numpy as np
import random
import time
import matplotlib.pyplot as plt
import copy as cp
import pickle
import pandas as pd

import RandomPEPS as rpeps
import StructureMatrixGenerator as smg
import trivialSimpleUpdate as tsu
import DoubleEdgeFactorGraphs as defg
import SimpleUpdate as su
import bmpslib as bmps

In [2]:
########################################################################################################################
#                                                                                                                      #
#                                    TRIVIAL SIMPLE UPDATE (TSU) ON RANDOM PEPS                                        #
#                                                                                                                      #
########################################################################################################################
np.random.seed(1)

# tSU parameters
N, M = 5, 5
bc = 'open'
dw = 1e-6
D_max = 3
t_max = 100
epsilon = 1e-10
dumping = 0.2
iterations = 30
d = 2
smat, _ = smg.finitePEPSobcStructureMatrixGenerator(N, M)
tensors, weights = smg.randomTensornetGenerator(smat, d, D_max)
n, m = smat.shape

In [ ]:
# constructing the dual double-edge factor graph
graph = defg.defg()
graph = su.TNtoDEFGtransform(graph, tensors, weights, smat)
graph.sumProduct(t_max, epsilon, dumping, printTime=1)
graph.calculateFactorsBeliefs()

### TODO ###
# define termination criteria using 2 site RDM
# check that tsu tensors weights list are disjoint
# compare BP and tsu for parallel and series update

In [3]:
# calculate the double site rdm
tSU_rdm = []
for i in range(m):
    tSU_rdm.append(tsu.doubleSiteRDM(i, tensors, weights, smat))

In [4]:
# trivial SU run
for i in range(iterations):
    tensors_next, weights_next = tsu.trivialsimpleUpdate(tensors,
                                                         weights,
                                                         smat,
                                                         D_max)
    ATD = 0
    tSU_rdm_next = []
    for j in range(m):
        tSU_rdm_next.append(tsu.doubleSiteRDM(i, tensors_next, weights_next, smat))
        ATD += tsu.traceDistance(tSU_rdm_next[j], tSU_rdm[j])
        tSU_rdm[j] = tSU_rdm_next[j]
    ATD /= m
    if ATD < dw:
        print('The ATD is: {} at iteration {}'.format(ATD, i))
        tensors = tensors_next
        weights = weights_next
        break
    tensors = tensors_next
    weights = weights_next

/Users/royelkabetz/Git/QBPmodule/trivialSimpleUpdate.py:326: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  leftDim = np.prod(shape[[leftIdx]])
/Users/royelkabetz/Git/QBPmodule/trivialSimpleUpdate.py:327: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  rightDim = np.prod(shape[[rightIdx]])


The ATD is: 1.7437484622443036e-08 at iteration 12


In [5]:
'''
# constructing the dual double-edge factor graph
graph = defg.defg()
graph = su.TNtoDEFGtransform(graph, tensors_tsu, weights_tsu, smat)
s = time.time()
graph.sumProduct(t_max, epsilon, dumping, printTime=1)
tot = time.time() - s
graph.calculateFactorsBeliefs()


tSU_1rdm = []
for i in range(n):
    tSU_1rdm.append(su.singleSiteRDM(i, tensors_tsu, weights_tsu, smat))

plt.figure()
plt.plot(range(len(errors_tsu)), errors_tsu)
plt.show()
'''

'\n# constructing the dual double-edge factor graph\ngraph = defg.defg()\ngraph = su.TNtoDEFGtransform(graph, tensors_tsu, weights_tsu, smat)\ns = time.time()\ngraph.sumProduct(t_max, epsilon, dumping, printTime=1)\ntot = time.time() - s\ngraph.calculateFactorsBeliefs()\n\n\ntSU_1rdm = []\nfor i in range(n):\n    tSU_1rdm.append(su.singleSiteRDM(i, tensors_tsu, weights_tsu, smat))\n\nplt.figure()\nplt.plot(range(len(errors_tsu)), errors_tsu)\nplt.show()\n'

In [6]:
'''
# calculate two site RDMs
tSU_2rdm = []
tSU_2rdmMess = []
for i in range(m):
    tSU_2rdm.append(tsu.doubleSiteRDM(i, tensors_tsu, weights_tsu, smat))
    tSU_2rdmMess.append(tsu.BPdoubleSiteRDM1(i, tensors_tsu, weights_tsu, smat, graph.messages_n2f))

ATD_tsu_Mess = 0
for i in range(m):
    ATD_tsu_Mess += tsu.traceDistance(tSU_2rdm[i], tSU_2rdmMess[i])
print(ATD_tsu_Mess / m)


'''
    

'\n# calculate two site RDMs\ntSU_2rdm = []\ntSU_2rdmMess = []\nfor i in range(m):\n    tSU_2rdm.append(tsu.doubleSiteRDM(i, tensors_tsu, weights_tsu, smat))\n    tSU_2rdmMess.append(tsu.BPdoubleSiteRDM1(i, tensors_tsu, weights_tsu, smat, graph.messages_n2f))\n\nATD_tsu_Mess = 0\nfor i in range(m):\n    ATD_tsu_Mess += tsu.traceDistance(tSU_2rdm[i], tSU_2rdmMess[i])\nprint(ATD_tsu_Mess / m)\n\n\n'

In [7]:
8.897045975742932e-05

8.897045975742932e-05